<a href="https://colab.research.google.com/github/taekyungss/computer_vision_planting_grass/blob/main/VAE_pytorch_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# prerequisites
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image


In [ ]:
bs = 100

# MNIST dataset
train_dataset = datasets.MNIST(root="./mnist_data/", train=True, transform = transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform = transforms.ToTensor(), download=False)

# Data Loader ( input pipeline )
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = bs, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = bs, shuffle = False)

In [ ]:
class VAE(nn.Module):
  def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
    super(VAE, self).__init__()

    # encoder part
    self.fc1 = nn.Linear(x_dim, h_dim1)
    self.fc2 = nn.Linear(h_dim1, h_dim2)
    self.fc31 = nn.Linear(h_dim2, z_dim)
    self.fc32 = nn.Linear(h_dim2, z_dim)

    # decoder part
    self.fc4 = nn.Linear(z_dim, h_dim2)
    self.fc5 = nn.Linear(h_dim2, h_dim1)
    self.fc6 = nn.Linear(h_dim1, x_dim)

  def encoder(self,x):
    h = F.relu(self.fc1(x))
    h = F.relu(self.fc2(h))
    return self.fc31(h), self.fc32(h)


  def sampling(self,mu,log_var):
    std = torch.exp(0.5*log_var)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

  def decoder(self,z):
    h = F.relu(self.fc4(z))
    h = F.relu(self.fc5(h))
    return F.sigmoid(self.fc6(h))

  def forward(self,x):
    mu, log_var = self.encoder(x.view(-1, 784))
    z = self.sampling(mu, log_var)
    return self.decoder(z), mu, log_var


vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim = 2)
if torch.cuda.is_available():
  vae.cuda()

In [ ]:
vae

In [ ]:
optimizer = optim.Adam(vae.parameters())

def loss_function(recon_x, x, mu, log_var):
  BCE = F.binary_cross_entropy(recon_x, x.view(-1,784), reduction='sum')
  # binary cross entropy loss
  KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
  # KLD loss => KL divergence loss
  return BCE + KLD

In [ ]:
def train(epoch):
  vae.train()
  train_loss =0
  for batch_idx, (data, _) in enumerate(train_loader):
    data = data.cuda()
    optimizer.zero_grad()

    recon_batch, mu, log_var = vae(data)
    loss = loss_function(recon_batch, data,mu, log_var)

    loss.backward()
    train_loss += loss.item()
    optimizer.step()

    if batch_idx % 100 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item() / len(data)))
  print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test():
  vae.eval()
  test_loss =0
  with torch.no_grad():
    for data, _ in test_loader:
      data = data.cuda()
      recon, mu, log_var = vae(data)

      test_loss += loss_function(recon, data, mu, log_var).item()

    test_loss /= len(test_loader.dataset)
    print("=====> Test set loss: {:.4f}".format(test_loss))

In [ ]:
for epoch in range(1,10):
  train(epoch)
  test()